In [2]:
%cd ..

C:\Users\georg\Documents\msc-projcet


In [3]:
import pandas as pd
from src.data.emails import clean_content

In [4]:
docs_df = pd.read_pickle("data/raw/webtrans/documents.pkl")
events_df = pd.read_pickle("data/raw/webtrans/events.pkl")
emails_df = pd.read_pickle("data/raw/webtransport/emails.pkl")

In [12]:
docs_df.head()

,id,name,title,timestamp,rfc_number,expires,revision
0,90872,draft-vvv-webtransport-overview,The WebTransport Protocol Framework,2020-04-15 13:49:33,None,2020-05-06T15:41:29,01
1,90874,draft-vvv-webtransport-http3,WebTransport over HTTP/3,2021-02-15 13:34:23,None,2021-07-31T15:53:13,03
2,95552,draft-kinnear-webtransport-http2,WebTransport using HTTP/2,2021-06-17 17:52:38,None,2021-08-26T14:42:26,02
3,96175,draft-ietf-webtrans-overview,The WebTransport Protocol Framework,2021-07-28 18:00:34,None,2022-01-29T18:00:34,02
4,100488,draft-ietf-webtrans-http3,WebTransport over HTTP/3,2021-05-19 13:13:57,None,2021-11-20T13:13:57,01


In [13]:
events_df.head()

,id,doc_id,type,timestamp,description
0,664621,90872,new_submission,2019-05-03 00:59:47,Uploaded new revision
1,664622,90872,new_submission,2019-05-03 01:00:37,Request for posting confirmation emailed to s...
2,664623,90872,new_submission,2019-05-03 01:00:56,New version approved
3,664624,90872,new_revision,2019-05-03 01:00:56,New version available: <b>draft-vvv-webtranspo...
4,714723,90872,new_submission,2019-11-03 15:41:18,Uploaded new revision


In [15]:
emails_df.head()

,timestamp,from,subject,content
0,2019-07-29 16:58:23,Victor Vasiliev <vasilvv@google.com>,[Webtransport] WebTransport Mailing List,"b'Hello everyone,\r\n\r\nJudging from the in-p..."
1,2019-07-29 23:28:57,Eliot Lear <lear@cisco.com>,Re: [Webtransport] WebTransport Mailing List,"b'Hi Victor,\r\n\r\nSorry if this has been cov..."
2,2019-07-29 17:32:30,Victor Vasiliev <vasilvv@google.com>,Re: [Webtransport] WebTransport Mailing List,"b'Hi Eliot,\r\n\r\nWebTransport is roughly ""We..."
3,2019-07-30 03:53:32,"""Martin J. Dürst"" <duerst@it.aoyama.ac.jp>",Re: [Webtransport] [hybi] WebTransport Mailing...,"b'I think Eliot\'s question shows that ""WebTra..."
4,2019-07-30 09:58:11,Lennart Grahl <lennart.grahl@gmail.com>,Re: [Webtransport] [hybi] WebTransport Mailing...,"b'Hi,\r\n\r\nI don\'t think what Victor said i..."


In [5]:
emails_df['content'] = emails_df['content'].apply(clean_content)

In [6]:
emails_df['content'][9]

b' Could you find a moment in your busy schedule to respond to what I wrote on this list before making your own new and exciting list, esp wrt this should find a way to target h2? -Andy'